In [1]:
from gensim.models import Word2Vec
from tensorflow import keras
import tensorflow as tf
from keras.models import load_model
import time
import glob
import os
import os.path
import random
import sys
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop

sys.path.append('..')
sys.path.append('../..')

Using TensorFlow backend.


In [2]:
#単語の分散表現のベクトルの次元数
vecSize = 15

In [24]:
model = Word2Vec.load("context2Vec.model")

In [26]:
print(len(model.wv.vocab))

print(model.wv.vocab['require'].__dict__)

print(model.wv.vocab)



11056
{'count': 4418, 'index': 65, 'sample_int': 4294967296}
{'on_ignored_nl': <gensim.models.keyedvectors.Vocab object at 0x7fcd71863518>, 'require': <gensim.models.keyedvectors.Vocab object at 0x7fcd71863358>, 'on_tstring_beg': <gensim.models.keyedvectors.Vocab object at 0x7fcd71863320>, 'on_tstring_content': <gensim.models.keyedvectors.Vocab object at 0x7fcd718632e8>, 'on_tstring_end': <gensim.models.keyedvectors.Vocab object at 0x7fcd718632b0>, 'on_nl': <gensim.models.keyedvectors.Vocab object at 0x7fcd71863208>, 'UNK': <gensim.models.keyedvectors.Vocab object at 0x7fcd71863198>, 'on_gvar': <gensim.models.keyedvectors.Vocab object at 0x7fcd718630f0>, 'on_op': <gensim.models.keyedvectors.Vocab object at 0x7fcd71863048>, 'true': <gensim.models.keyedvectors.Vocab object at 0x7fcd71832400>, 'rescue': <gensim.models.keyedvectors.Vocab object at 0x7fcd71832438>, 'on_const': <gensim.models.keyedvectors.Vocab object at 0x7fcd71832ac8>, 'puts': <gensim.models.keyedvectors.Vocab object at 0x

In [5]:
#学習データのファイル群を取得
file = glob.glob("../../src/repositories2TokenWithComment/all.txt")
print(file)

['../../src/repositories2TokenWithComment/all.txt']


In [6]:
context = []
f = open(file[0])
lines = f.readlines()
for line in lines:
    temp_data = line.replace('\n','')
    temp_data = temp_data.split(' ')[0:20]
    context.append(temp_data)
f.close()
print(len(context))

print(context[0])
print(context[0][0])

48920
['require', 'on_tstring_beg', 'on_tstring_content', 'on_tstring_end', 'require', 'on_tstring_beg', 'on_tstring_content', 'on_tstring_end', 'require', 'on_tstring_beg', 'on_tstring_content', 'on_tstring_end', 'require', 'on_tstring_beg', 'on_tstring_content', 'on_tstring_end', 'require', 'on_tstring_beg', 'on_tstring_content', 'on_tstring_end']
require


In [7]:
#テキストのベクトル化
#context2Vec 20トークンに区切った配列を格納している。
context2Vec = []

for i in range(len(context)):
    temp_ary = []
    for j in range(len(context[i])):
        temp_ary.append(np.array(model.wv[context[i][j]]))
    temp_ary = np.array(temp_ary)
    context2Vec.append(temp_ary)
print(context2Vec[0][0])

[ 1.6336491  -1.1472334   3.687609   -3.0286171  -0.78851205 -2.1730828
 -7.614615   -0.46730554 -2.099985    3.9699225  -0.3383838  -2.319487
 -3.6048248   3.3231404   0.38078317]


In [158]:
print(len(context2Vec))

50370


In [159]:
maxLen = 19

In [160]:
sentences = []
next_chars = []

In [161]:
for i in range(len(context2Vec)):
    sentences.append(context2Vec[i][0:maxLen])
    next_chars.append(context2Vec[i][-1])

In [162]:
print(len(sentences[0][0]))

15


In [163]:
# モデルを定義する

In [164]:
models = Sequential()
models.add(LSTM(128,input_shape=(maxLen,vecSize)))

In [165]:
models.add(Dense(vecSize))

In [166]:
models.add(Activation('softmax'))

In [167]:
optimizer = RMSprop(lr=0.01)

In [168]:
#損失関数は平均２条誤差を使用

models.compile(loss='mean_squared_error', optimizer=optimizer)

In [169]:
def sample(preds):
    return model.most_similar( [ np.array(preds) ], [], 5)[0][0]

In [170]:
for iteration in range(1,30):
    print()
    print('-'*50)
    print('繰り返し回数: ', iteration)
    models.fit([sentences],[next_chars],batch_size=128,epochs=2)

    start_index = random.randint(0, len(context)-maxLen-1)

    generated = ''
    sentence = model.most_similar([sentences[start_index][0] ],[],1)[0][0]
    generated += sentence
    print('-----Seedを生成しました"' + sentence + '"')

    sys.stdout.write(generated)

    x = sentences[start_index]
    print(x)
    for i in range(400):
        input_x = x[-maxLen:]
        input_x = np.array(input_x)
        input_x = np.array([input_x])
        preds = models.predict([input_x],verbose=2)[0]
        next_char = sample(preds)
        generated += next_char
        generated += " "
        sentence = sentence[1:] + next_char
        x = np.append(x, np.array([model.wv[next_char]]),axis=0)

        sys.stdout.write(next_char)
        sys.stdout.write(" ")
        sys.stdout.flush()
    print()
    


--------------------------------------------------
繰り返し回数:  1
Epoch 1/2
50370/50370 [==============================] - 7s 134us/step - loss: 6.2838
Epoch 2/2
50370/50370 [==============================] - 6s 127us/step - loss: 6.2799
-----Seedを生成しました"triggers"
triggers[[-2.5423765e-02 -1.5988568e-02 -3.1538960e-02 -2.3255661e-02
  -1.7029686e-02  7.7109495e-03  2.6160423e-02 -1.1852775e-02
   2.9564898e-02 -1.5287434e-02  3.1372204e-02  2.1353972e-03
   1.6890986e-02  5.0895228e-03 -3.2773342e-02]
 [ 6.5971565e+00 -1.1656797e+00  4.9633188e+00  3.0746913e+00
   1.3902647e+00 -3.5046384e+00 -2.7392197e-01  3.3865619e+00
  -3.9492917e+00  2.5930184e-01  6.0250866e-01 -1.8071731e+00
  -2.4979098e+00  5.2913389e+00 -7.8390634e-01]
 [ 2.8311422e+00  2.7601938e+00 -3.0563632e-01  2.2960658e+00
  -1.4571580e+00  5.6545049e-01 -1.8245670e+00 -4.0462326e-02
  -1.6883502e+00  3.2093949e+00 -9.6010041e-01 -1.6259836e+00
  -1.9690000e-01 -1.2828954e+00 -2.7463770e+00]
 [ 6.4574451e+00  3.2943847e

/Users/makoto/anaconda3/envs/ruby_rnn/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.
/Users/makoto/anaconda3/envs/ruby_rnn/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr

on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr

on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr

...

on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr

on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr

detail on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_

   1.3125958e-02 -3.2134987e-02  1.7320987e-02]]
on_embexpr_beg on_embexpr_beg on_embexpr_beg def def def on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg metachecks on_embexpr_beg access_path on_embexpr_beg access_path on_embexpr_beg access_path on_embexpr_beg def def def def def on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg def def on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexp

def def def on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_beg on_embexpr_be

In [171]:
models.save("context2Seq.model")

In [172]:
#ここからはmodelをloadしてきて遊んでみる


In [173]:
models = keras.models.load_model("context2Seq.model")
print(np.array([context2Vec[0][:19]]).shape)

(1, 19, 15)


In [178]:
preds = models.predict(np.array([context2Vec[1][:19]]),verbose=2)

1/1 - 0s


In [177]:
preds[0]

array([5.3990453e-01, 6.2659581e-07, 4.6009478e-01, 1.7984304e-12,
       2.6135183e-16, 8.4461963e-17, 2.1367236e-16, 5.7549324e-16,
       2.1268418e-17, 1.3251130e-09, 1.9510309e-16, 3.9333186e-17,
       5.7746776e-17, 9.8142480e-11, 8.5076353e-17], dtype=float32)